In [1]:
# Standard library imports
from pathlib import Path
import os
import sys
import math
import itertools
from typing import List, Union, Sequence


# Third-party library imports
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from numpy import ndarray
from tqdm import tqdm
from scipy.spatial import distance
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from PIL import Image
from torch_geometric.data import Data
from torch_geometric.nn import ChebConv
from torch_geometric.nn.conv import GeneralConv

# Project-specific import
from Utils_GAN.classes import DynamicGraphTemporalSignal, GConvGRU, BoidDatasetLoader,RecurrentGCN
from Utils_GAN.classes import Encoder,Decoder,GraphSeqGenerator
from Utils_GAN.classes import GraphSeqDiscriminator,GraphSeqGenerator


from Utils_GAN.subfunctions import temporal_signal_split, train,plot_predictions,test,create_video_from_images
from Utils_GAN.subfunctions import train_generator,test_generator,test_generator_plot

### For Now: Grabbing Parts of Pytorch Geometric Temporal As Having Difficulty Setting Up The Environment For It

## Set Up BoidDatasetLoader To Load Entire Dataset
Following Example of EnglandCovidDatasetLoader from Pytorch Geometric Temporal

In [2]:
class BoidDatasetLoader(object):
    """
    The BoidDatasetLoader class is designed to load and process the Boid Dataset.
    It reads data from CSV files, processes it, and prepares it for further analysis or modeling. The class performs the
    following key functions:

    1. Initialization (__init__ method):
       - Calls the _read_data method to load and preprocess the data.

    2. Data Reading and Preprocessing (_read_data method):
       - Reads simulation data from 'simulation.csv' and 'simulation_edges.csv'.
       - Drops the 'Simulation' column from both dataframes.
       - Renames columns to standardize the naming convention (e.g., 'Boids' to 'BoidID', 'Boid_i' to 'BoidID_i').
       - Stores the cleaned dataframes in the _dataset attribute.
       - Calls the _process_dataset method to further process the data and extract features, edges, and edge weights.

    3. Dataset Processing (_process_dataset method):
       - Groups the simulation data by 'Timestep'.
       - Initializes lists to store edge indices, node features, and distances.
       - Iterates over each timestep to extract relevant data for that timestep.
       - Converts the dataframes to NumPy arrays for efficient computation.
       - Creates a dictionary to map BoidID to coordinates.
       - Uses vectorized operations to calculate distances between boids with edges.

    4. Getting Features and Edge Weights (_get_edge_weights() and _get_features() methods):
        - Normalized features and _edge_weights using min-max normalization
        - Attributes are now normalized when called and returned

    5. Getting Target (_get_target() method):
        - If t is the current index of our dataset, then t+1 is the target
        - Contains the node features of the graph at t+1
        - Will probably not used this as wouldnt work entirely for more than 1 timestep prediction

    Attributes:
        _dataset: A tuple containing the cleaned simulation data and edge data.
        features: Node features extracted from the simulation data.
        _edges: Edge indices representing connections between boids.
        _edge_weights: Weights of the edges, which could represent distances or other metrics.

    Methods:
        __init__(): Initializes the class and reads the data.
        _read_data(): Reads and preprocesses the data from CSV files.
        _process_dataset(sim_df, sim_edges_df): Processes the dataset to extract features, edges, and edge weights.
    """
    def __init__(self):
        self._read_data()

    def _read_data(self):
        path_to_sim = '../data/simulation.csv'
        path_to_sim_edges = '../data/simulation_edges.csv'
        sim_df = pd.read_csv(path_to_sim)
        sim_edges_df = pd.read_csv(path_to_sim_edges)

        sim_df.drop(columns='Simulation', inplace=True)
        sim_df.rename(columns={'Boids':'BoidID'}, inplace=True)

        sim_edges_df.drop(columns='Simulation', inplace=True)
        sim_edges_df.rename(columns={'Boid_i':'BoidID_i', 'Boid_j':'BoidID_j'}, inplace=True)
        
        self._dataset = (sim_df, sim_edges_df)
        self.features, self._edges, self._edge_weights = self._process_dataset(self._dataset[0], self._dataset[1])

    def _process_dataset(self, sim_df, sim_edges_df):
        # Group the dataframes by 'Timestep'
        sim_grouped = sim_df.groupby('Timestep')
        edges_grouped = sim_edges_df.groupby('Timestep')
        
        # Initialize lists to store edge indices and node features
        edge_indices = []
        node_features = []
        
        distances = []

        # Iterate over each group
        for timestep, _ in sim_grouped:
            # Extract relevant columns for the current timestep
            timestep_df = sim_grouped.get_group(timestep)[['x', 'y', 'dx', 'dy', 'BoidID']]
            timestep_edges_df = edges_grouped.get_group(timestep)[['BoidID_i', 'BoidID_j']]
            
            # Convert dataframes to numpy arrays
            node_array = timestep_df[['x', 'y', 'dx', 'dy']].to_numpy()
            edge_array = timestep_edges_df.to_numpy().T

            # Create a dictionary to map BoidID to coordinates
            boid_coords = {boid_id: coords for boid_id, coords in zip(timestep_df['BoidID'], timestep_df[['x', 'y']].values)}
            
            # Get coordinates for boids involved in edges
            boid_i_coords = np.array([boid_coords[boidid_i] for boidid_i in edge_array[0]])
            boid_j_coords = np.array([boid_coords[boidid_j] for boidid_j in edge_array[1]])
            
            # Calculate distances using vectorized operations
            timestep_distances = np.linalg.norm(boid_i_coords - boid_j_coords, axis=1)
            
            distances.append(timestep_distances)

            # Append the numpy arrays to the respective lists
            edge_indices.append(edge_array)
            node_features.append(node_array)

        # Return the lists of edge indices and node features
        return node_features, edge_indices, distances

    def _compute_feature_min_max(self, feature_list):
        """
        Compute the minimum and maximum of features across all node features.

        Parameters:
        all_node_features (list of np.ndarray): List of numpy arrays containing node features.

        Returns:
        tuple: A tuple containing two numpy arrays: (final_min, final_max).
        """
        if not feature_list:
            raise ValueError("The input list 'all_node_features' is empty.")
        
        # Initialize final_min and final_max with appropriate dimensions
        if len(feature_list[0].shape) == 0:
            raise ValueError('The input list is missing node features') 
        
        if len(feature_list[0].shape) == 1:
            feature_dim = 1
            axis_val = None
            final_min = float('inf')
            final_max = float('-inf')
        else:
            feature_dim = feature_list[0].shape[1]
            axis_val = 0
            final_min = np.array([float('inf')] * feature_dim)
            final_max = np.array([float('-inf')] * feature_dim)
        
        # Iterate through all node features to compute final_min and final_max
        for features in feature_list:
            curr_max = np.max(features, axis=axis_val)
            curr_min = np.min(features, axis=axis_val)
            final_max = np.max(np.array([final_max, curr_max]), axis=0)
            final_min = np.min(np.array([final_min, curr_min]), axis=0)
        
        return final_min, final_max
    
    def _minmax_scale(self, feature_list, final_min, final_max):
        normalized = []
        for i in range(len(feature_list)):
            X = feature_list[i]
            X_std = (X - final_min) / (final_max - final_min)
            normalized.append(X_std)
        return normalized

    def undo_minmax_scale(self, normalized_feature_list, final_min, final_max):
        unnormalized = []
        for i in range(len(normalized_feature_list)):
            X_std = normalized_feature_list[i]
            X_scaled = X_std * (final_max - final_min) + final_min
            unnormalized.append(X_scaled)
        return unnormalized
    

    def _get_edge_weights(self):
        self.min_edge_weight, self.max_edge_weight = self._compute_feature_min_max(self._edge_weights)
        self._edge_weights = self._minmax_scale(self._edge_weights, self.min_edge_weight, self.max_edge_weight)

    def _get_features(self):
        self.min_features, self.max_features = self._compute_feature_min_max(self.features)
        self.features = self._minmax_scale(self.features, self.min_features, self.max_features)
    def _get_targets(self):
        self.targets = [self.features[i] for i in range(1, len(self.features))]


    def get_dataset(self) -> DynamicGraphTemporalSignal:

        self._get_edge_weights()
        self._get_features()
        self._get_targets()
        dataset = DynamicGraphTemporalSignal(
            self._edges[:len(self.features)-1], 
            self._edge_weights[:len(self.features)-1], 
            self.features[:len(self.features)-1], 
            self.targets
        )
        return dataset

In [ ]:
# reset kernel if not working
loader = BoidDatasetLoader()
dataset = loader.get_dataset()
dataset.snapshot_count

In [ ]:
loader.min_features[0:2], loader.max_features[0:2]

# Split Dataset Into Training and Testing

In [5]:
# Again from https://github.com/benedekrozemberczki/pytorch_geometric_temporal/blob/master/torch_geometric_temporal/signal/train_test_split.py

def temporal_signal_split(data_iterator, train_ratio=0.8):
    train_snapshots = int(data_iterator.snapshot_count * train_ratio)
    train_iterator = data_iterator[0:train_snapshots]
    test_iterator = data_iterator[train_snapshots:]
    return train_iterator, test_iterator

In [ ]:
train_dataset, test_dataset = temporal_signal_split(dataset)

train_dataset.snapshot_count, test_dataset.snapshot_count

## Create GConvGRU For Recurrent Layer In Our GNN

## Basic Graph Recurrent Neural Network

In [ ]:
#  Training a basic model for tesiting purpose
model = RecurrentGCN(node_features=4, filters=32)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()
for epoch in range(3):
    for seq_start in tqdm(range(0, train_dataset.snapshot_count - 5, 5)):
        h_t_prev = None
        for i in range(5):
            snapshot = train_dataset[seq_start+i]
            y_hat, h_t = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr, h_t_prev)
            h_t_prev = h_t
        cost = F.mse_loss(y_hat, snapshot.y)
        cost.backward()
        optimizer.step()
        optimizer.zero_grad()


## Training

In [ ]:
model = RecurrentGCN(node_features=4, filters=32)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train(train_dataset, 10, model, optimizer)

In [ ]:
test(test_dataset, model,loader=loader)

In [ ]:
import cv2
import os
import re

# Example usage
#create_video_from_images('../generator_test_plots_at_epoch_10', 'generator_test_plots_at_epoch_10.2.mp4', fps=30)

In [6]:
#create_video_from_images('../generator2_test_plots_at_epoch_30', 'generator2_test_plots_at_epoch_30.mp4', fps=30)

## Working on GAN [Generator: (Encoder, Decoder)] [Discriminator: (Encoder)]

In [15]:
class GraphSeqGenerator(torch.nn.Module):
    def __init__(self, node_feat_dim, enc_hidden_dim, enc_latent_dim, dec_hidden_dim, pred_horizon, min_max_x, min_max_y, min_max_edge_weight, visualRange, device):
        super(GraphSeqGenerator, self).__init__()
        self.encoder = Encoder(node_feat_dim, enc_hidden_dim, enc_latent_dim)
        self.decoder = Decoder(enc_latent_dim, dec_hidden_dim, node_feat_dim)
        self.out_steps = pred_horizon
        self.min_x, self.max_x = min_max_x
        self.min_y, self.max_y = min_max_y
        self.min_edge_weight, self.max_edge_weight = min_max_edge_weight
        self.visualRange = visualRange

        self.device = device

    def _compute_edge_index_and_weight(self, y_hat):
        # Not designed for batches :/
        # Grab x and y features
        y_hat_x = y_hat[:, 0].detach() # originally I moved these to cpu but now it depends on the 'device'
        y_hat_y = y_hat[:, 1].detach()

        # Undo normalization
        y_hat_x = y_hat_x * (self.max_x - self.min_x) + self.min_x
        y_hat_y = y_hat_y * (self.max_y - self.min_y) + self.min_y

        # Compute the distance of all points and include that edge if its less than visualRange
        coords = torch.stack((y_hat_x, y_hat_y), dim=1)
        distances = torch.cdist(coords, coords, p=2)

        # Get indices where distance is less than visualRange
        edge_indices = torch.where((distances < self.visualRange) & (distances > 0)) # returns a tuple with indices
        
        # Create edge_index and edge_attr
        edge_index = torch.vstack((edge_indices[0], edge_indices[1]))
        edge_weight = distances[edge_indices]

        #Normalize edge_weight
        edge_weight = (edge_weight - self.min_edge_weight) / (self.max_edge_weight - self.min_edge_weight)
        
        edge_index = edge_index.to(device=self.device)
        edge_weight = edge_weight.to(device=self.device)

        return edge_index, edge_weight


        
    def forward(self, sequence, h_enc, h_dec):
        # Warmup Section
        for i in range(sequence.snapshot_count):
            snapshot = sequence[i]
            snapshot_x = snapshot.x.to(self.device)
            snapshot_edge_index = snapshot.edge_index.to(self.device)
            snapshot_edge_attr = snapshot.edge_attr.to(self.device)

            z, h_enc_0 = self.encoder(snapshot_x, snapshot_edge_index, snapshot_edge_attr, h_enc)
            # This part doesnt have the random vector in the hidden state so I need to add it hear 
            y_hat, h_dec_0 = self.decoder(z, snapshot_edge_index, snapshot_edge_attr, h_dec)

            h_enc = h_enc_0
            h_dec = h_dec_0

        y_hat_features = []
        y_hat_edge_indices = []
        y_hat_edge_attrs = []
        y_hat_features.append(y_hat)
        y_hat_edge_index, y_hat_edge_attr = self._compute_edge_index_and_weight(y_hat)
        y_hat_edge_indices.append(y_hat_edge_index)
        y_hat_edge_attrs.append(y_hat_edge_attr)
        
        # Prediction Section
        for _ in range(self.out_steps-1):
            y_hat_edge_index, y_hat_edge_attr = self._compute_edge_index_and_weight(y_hat)

            z, h_enc_0 = self.encoder(y_hat, y_hat_edge_index, y_hat_edge_attr, h_enc)
            y_hat, h_dec_0 = self.decoder(z, y_hat_edge_index, y_hat_edge_attr, h_dec)

            y_hat_features.append(y_hat)
            y_hat_edge_index, y_hat_edge_attr = self._compute_edge_index_and_weight(y_hat)
            y_hat_edge_indices.append(y_hat_edge_index)
            y_hat_edge_attrs.append(y_hat_edge_attr)
            
        # Need to save it in cpu
        y_hat_edge_indices = [y_hat_edge_index.cpu() for y_hat_edge_index in y_hat_edge_indices]
        y_hat_edge_attrs = [y_hat_edge_attr.cpu() for y_hat_edge_attr in y_hat_edge_attrs]
        y_hat_features = [y_hat_feature.cpu() for y_hat_feature in y_hat_features]

        y_hat_seq = DynamicGraphTemporalSignal(y_hat_edge_indices, y_hat_edge_attrs, y_hat_features, [None]*self.out_steps)
        return y_hat_features, y_hat_seq

In [16]:
class GraphSeqDiscriminator(torch.nn.Module):
    def __init__(self, node_feat_dim, enc_hidden_dim, enc_latent_dim, pred_horizon, device):
        super(GraphSeqDiscriminator, self).__init__()

        self.encoder = Encoder(node_feat_dim, enc_hidden_dim, enc_latent_dim)
        self.linear = torch.nn.Linear(enc_latent_dim, 1)
        self.out_steps = pred_horizon
        self.device = device

    
    def forward(self, sequence, h_enc, shouldDetach=False):
        for i in range(sequence.snapshot_count):
            snapshot = sequence[i]
            snapshot_x = snapshot.x.to(self.device)
            snapshot_edge_index = snapshot.edge_index.to(self.device)
            snapshot_edge_attr = snapshot.edge_attr.to(self.device)

            if shouldDetach:
                snapshot_x = snapshot_x.detach()
                snapshot_edge_index = snapshot_edge_index.detach()
                snapshot_edge_attr = snapshot_edge_attr.detach()

            z, h_enc_0 = self.encoder(snapshot_x, snapshot_edge_index, snapshot_edge_attr, h_enc)
            h_enc = h_enc_0
        
        z = F.relu(z)

        # Apply global mean pooling across the node dimension (dim=0) to aggregate node features
        z_pooled = z.mean(dim=0)
        out = self.linear(z_pooled)
        out = torch.sigmoid(out)

        return out, h_enc_0

In [17]:
def train_gan(train_data, num_epochs, generator, discriminator, criterion, optimizerG, optimizerD, device, k=1, window=8, delay=0, horizon=1, stride=1):
    """
    Trains the given model using the provided training data.

    Args:
        train_data (Dataset): The dataset containing the training data.
        num_epochs (int): The number of epochs to train the model.
        model (nn.Module): The model to be trained.
        optimizer (Optimizer): The optimizer used for training the model.
        window (int, optional): The size of the input sequence window. Defaults to 8.
        delay (int, optional): The delay between the input sequence and the target sequence. Defaults to 0.
        horizon (int, optional): The prediction horizon. Defaults to 1.
        stride (int, optional): The stride for iterating over the training data. Defaults to 1.

    Returns:
        None
    """
    total_timesteps = train_data.snapshot_count
    sample_span = window + delay + horizon
    

    generator.train()
    for epoch in range(num_epochs):
        print(f'Epoch: {epoch+1}/{num_epochs}')
        epoch_cost = 0
        for start in tqdm(range(0, total_timesteps - sample_span + 1, stride), desc='Training'):
            input_seq = train_data[start:start + window]
            y_seq = train_data[start + window + delay: start + window + delay + horizon]


            y_hat_seq_feats, y_hat_seq = generator(input_seq, None, None)


            # Discriminator Step
            discriminator.zero_grad()

            output_of_real, _ = discriminator(y_seq, h_enc=None)

            errD_real = criterion(output_of_real, torch.ones(1, device=device))

            errD_real.backward()

            output_of_fake, _ = discriminator(y_hat_seq, h_enc=None, shouldDetach=True)
            errD_fake = criterion(output_of_fake, torch.ones(1, device=device))

            errD_fake.backward()

            errD = errD_real + errD_fake

            optimizerD.step()

            # Generator Step
            generator.zero_grad()
            min_mse_loss = float('inf')
            best_y_hat_seq = None

            for _ in range(k):
                y_hat_seq_feats, y_hat_seq = generator(input_seq, None, None)
                output_of_fake2, _ = discriminator(y_hat_seq, h_enc=None)

                y_hat = torch.stack([y_hat_seq[i].x for i in range(y_hat_seq.snapshot_count)], dim=0).to(device=device)
                y_actual = torch.stack([y_seq[i].x for i in range(y_seq.snapshot_count)], dim=0).to(device=device)

                mse_loss = F.mse_loss(y_hat, y_actual)
                if mse_loss < min_mse_loss:
                    min_mse_loss = mse_loss
                    best_y_hat_seq = y_hat_seq

            output_of_fake2, _ = discriminator(best_y_hat_seq, h_enc=None)

            errG = criterion(output_of_fake2, torch.ones(1, device=device))

            errG += min_mse_loss

            errG.backward()

            optimizerG.step()         

In [ ]:
device = torch.device("cpu")
generator = GraphSeqGenerator(node_feat_dim=4,
                              enc_hidden_dim=32,
                              enc_latent_dim=16,
                              dec_hidden_dim=32,
                              pred_horizon=8,
                              min_max_x=(loader.min_features[0], loader.max_features[0]),
                              min_max_y=(loader.min_features[1], loader.max_features[1]),
                              min_max_edge_weight=(loader.min_edge_weight, loader.max_edge_weight),
                              visualRange=75,
                              device=device,
                            )
generator.to(device)

discriminator = GraphSeqDiscriminator(node_feat_dim=4,
                                      enc_hidden_dim=32,
                                      enc_latent_dim=16,
                                      pred_horizon=8,
                                      device=device)
discriminator.to(device)

criterion = torch.nn.BCELoss()
optimizerG = torch.optim.Adam(generator.parameters(), lr=0.001)
optimizerD = torch.optim.Adam(discriminator.parameters(), lr=0.001)
train_gan(train_dataset, 1, generator, discriminator, criterion, optimizerG, optimizerD, device, k=6, window=8, horizon=8 )